# queue

Readings:
 - https://docs.python.org/3/library/queue.html

## Basic Setup

In [1]:
import concurrent.futures
import logging
import queue
import threading
import time

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('[%(threadName)s]: %(message)s'))
logger.addHandler(handler)

## Without blocking

In [2]:
fifo = queue.Queue(2)


def thread(n):
    try:
        fifo.put_nowait(n)
    except queue.Full:
        logger.info("Failed to put %(n)s", {'n': n})
    else:
        logger.info(f'Add %(n)s', {'n': n})
    time.sleep(0.001)
    try:
        logger.info(f'Get %(get)s', {'get': fifo.get_nowait()})
    except queue.Empty:
        logger.info("Failed to get")


with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
     executor.map(thread, range(10))

logger.info('FIFO is empty? %(is_empty)s', {'is_empty': fifo.empty()})

[ThreadPoolExecutor-0_0]: Add 0
[ThreadPoolExecutor-0_1]: Add 1
[ThreadPoolExecutor-0_2]: Failed to put 2
[ThreadPoolExecutor-0_3]: Failed to put 3
[ThreadPoolExecutor-0_0]: Get 0
[ThreadPoolExecutor-0_4]: Add 4
[ThreadPoolExecutor-0_1]: Get 1
[ThreadPoolExecutor-0_1]: Add 6
[ThreadPoolExecutor-0_0]: Add 5
[ThreadPoolExecutor-0_3]: Get 5
[ThreadPoolExecutor-0_2]: Get 4
[ThreadPoolExecutor-0_2]: Add 8
[ThreadPoolExecutor-0_3]: Add 7
[ThreadPoolExecutor-0_1]: Get 7
[ThreadPoolExecutor-0_4]: Get 6
[ThreadPoolExecutor-0_0]: Get 8
[ThreadPoolExecutor-0_2]: Failed to get
[ThreadPoolExecutor-0_1]: Add 9
[ThreadPoolExecutor-0_3]: Get 9
[ThreadPoolExecutor-0_1]: Failed to get
[MainThread]: FIFO is empty? True


## Blocking

In [3]:
fifo = queue.Queue(2)


def thread(n):
    fifo.put(n)
    logger.info(f'Add %(n)s', {'n': n})
    time.sleep(0.001)
    logger.info(f'Get %(get)s', {'get': fifo.get()})


with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
     executor.map(thread, range(10))

logger.info('FIFO is empty? %(is_empty)s', {'is_empty': fifo.empty()})

[ThreadPoolExecutor-1_0]: Add 0
[ThreadPoolExecutor-1_1]: Add 1
[ThreadPoolExecutor-1_0]: Get 0
[ThreadPoolExecutor-1_2]: Add 2
[ThreadPoolExecutor-1_1]: Get 1
[ThreadPoolExecutor-1_3]: Add 3
[ThreadPoolExecutor-1_2]: Get 2
[ThreadPoolExecutor-1_4]: Add 4
[ThreadPoolExecutor-1_3]: Get 3
[ThreadPoolExecutor-1_0]: Add 5
[ThreadPoolExecutor-1_4]: Get 4
[ThreadPoolExecutor-1_1]: Add 6
[ThreadPoolExecutor-1_0]: Get 5
[ThreadPoolExecutor-1_2]: Add 7
[ThreadPoolExecutor-1_1]: Get 6
[ThreadPoolExecutor-1_3]: Add 8
[ThreadPoolExecutor-1_2]: Get 7
[ThreadPoolExecutor-1_4]: Add 9
[ThreadPoolExecutor-1_3]: Get 8
[ThreadPoolExecutor-1_4]: Get 9
[MainThread]: FIFO is empty? True


In [4]:
fifo = queue.Queue()

def worker():
    logger.info('Worker started')
    while True:
        logger.info('%(item)s', {'item': fifo.get()})
        fifo.task_done()
        
threading.Thread(target=worker, daemon=True).start()

for item in range(30):
    fifo.put(item)
logger.info('All task requests sent')

fifo.join()
logger.info('All work completed')

[Thread-4]: Worker started
[MainThread]: All task requests sent
[Thread-4]: 0
[Thread-4]: 1
[Thread-4]: 2
[Thread-4]: 3
[Thread-4]: 4
[Thread-4]: 5
[Thread-4]: 6
[Thread-4]: 7
[Thread-4]: 8
[Thread-4]: 9
[Thread-4]: 10
[Thread-4]: 11
[Thread-4]: 12
[Thread-4]: 13
[Thread-4]: 14
[Thread-4]: 15
[Thread-4]: 16
[Thread-4]: 17
[Thread-4]: 18
[Thread-4]: 19
[Thread-4]: 20
[Thread-4]: 21
[Thread-4]: 22
[Thread-4]: 23
[Thread-4]: 24
[Thread-4]: 25
[Thread-4]: 26
[Thread-4]: 27
[Thread-4]: 28
[Thread-4]: 29
[MainThread]: All work completed
